# ECE 284 Final Project - Software

## Vanilla Training VGGNet with 4 bit activation and weight quantisation

### Initialisation Code

In [2]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms

from models import *

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')

batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])

train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
    
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

### Chokepoint Block and SkipBlock Optimisation

In [3]:
# [SEE custom_vgg.py for details on SkipBlock optimisation]
from models.custom_vgg import SkipBlock

use_skip_block = False
if use_skip_block:
    print("Using SkipBlock optimisation")
    skip_block = SkipBlock()
    model.features[20] = skip_block
    for i in range(21, 33):
        model.features[i] = nn.Identity()
else:
    print("Not using SkipBlock optimisation")
    model.features[24] = QuantConv2d(256, 8, kernel_size=3, padding=1, bias=False)
    model.features[25] = nn.BatchNorm2d(8)
    model.features[26] = nn.ReLU(inplace=True)
    
    model.features[27] = QuantConv2d(8, 8, kernel_size=3, padding=1, bias=False)
    model.features[28] = nn.Identity()
    model.features[29] = nn.ReLU(inplace=True)
    
    model.features[30] = QuantConv2d(8, 512, kernel_size=3, padding=1, bias=False)
    model.features[31] = nn.BatchNorm2d(512)
    model.features[32] = nn.ReLU(inplace=True)

print(model)

Not using SkipBlock optimisation
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3

### Training Loop

In [3]:
lr = 1e-2
weight_decay = 1e-5
epochs = 150
best_prec = 0

model.cuda()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1).cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay, nesterov=True)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)

for epoch in range(0, epochs):
    # adjust_learning_rate(optimizer, epoch)
    train(trainloader, model, criterion, optimizer, epoch)
    scheduler.step()

    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.939 (0.939)	Data 0.420 (0.420)	Loss 2.4035 (2.4035)	Prec 11.719% (11.719%)
Epoch: [0][100/391]	Time 0.049 (0.056)	Data 0.001 (0.005)	Loss 1.7465 (2.0822)	Prec 39.844% (28.759%)
Epoch: [0][200/391]	Time 0.047 (0.052)	Data 0.001 (0.003)	Loss 1.5257 (1.8682)	Prec 50.781% (37.512%)
Epoch: [0][300/391]	Time 0.048 (0.050)	Data 0.001 (0.003)	Loss 1.4412 (1.7406)	Prec 54.688% (43.010%)
Validation starts
Test: [0/79]	Time 0.289 (0.289)	Loss 1.4663 (1.4663)	Prec 56.250% (56.250%)
 * Prec 53.660% 
best acc: 53.660000
Epoch: [1][0/391]	Time 0.213 (0.213)	Data 0.181 (0.181)	Loss 1.4423 (1.4423)	Prec 53.906% (53.906%)
Epoch: [1][100/391]	Time 0.047 (0.049)	Data 0.001 (0.004)	Loss 1.3009 (1.3072)	Prec 68.750% (63.157%)
Epoch: [1][200/391]	Time 0.047 (0.048)	Data 0.001 (0.003)	Loss 1.3568 (1.2779)	Prec 64.844% (64.754%)
Epoch: [1][300/391]	Time 0.046 (0.048)	Data 0.001 (0.002)	Loss 1.0507 (1.2527)	Prec 75.000% (66.100%)
Validation starts
Test: [0/79]	Time 0.184 (0.184)	Loss 1.

### Testing Loop

In [5]:
PATH = "result/VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

print(model)

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

/tmp/ipykernel_893700/128479599.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)



Test set: Accuracy: 9072/10000 (91%)



### Weight and Activation recovery verification

#### Getting the right target layer and attaching hooks for fwd pass

In [4]:
if use_skip_block:
    target_layer = model.features[20].conv27
    next_layer = model.features[20].bn28
else:
    target_layer = model.features[27]
    next_layer = model.features[28]

# some sanity checking to ensure we're extracting the right layer
assert isinstance(target_layer, QuantConv2d)

input_records = {}
def get_input_hook(name):
    def hook(model, input, output):
        input_records[name] = input[0].detach().clone()
    return hook

target_layer.register_forward_hook(get_input_hook('conv8x8_input'))
next_layer.register_forward_hook(get_input_hook('next_layer_input'))

# Running a sample forward pass to extract inputs
data, _ = next(iter(testloader))
data = data.cuda()
model = model.cuda()
model.eval()
with torch.no_grad():
    output = model(data)
conv8x8_input = input_records['conv8x8_input'].cpu().numpy()
print(f"Captured Transient Input for 8x8 layer: {conv8x8_input}")

target_layer = target_layer.cpu()

Captured Transient Input for 8x8 layer: [[[[0.7528543  1.7992511  1.7467965  1.3647989 ]
   [0.         0.26089427 0.         0.        ]
   [0.         0.05128749 0.6869564  0.42997885]
   [0.         0.         0.         0.        ]]

  [[0.12536542 1.0985982  0.97800106 0.8837508 ]
   [0.12086887 1.5830147  0.19548273 0.        ]
   [0.93421614 1.587489   0.32703155 0.        ]
   [0.         0.30065754 0.         0.        ]]

  [[0.         0.         0.         0.04210338]
   [0.         0.         0.         0.2601679 ]
   [0.         0.30907613 1.5854645  1.0959355 ]
   [0.         0.18303858 0.48659316 1.2263494 ]]

  ...

  [[0.         0.         0.         0.5830591 ]
   [0.         0.         0.56545293 1.357479  ]
   [0.         0.         0.         0.47578564]
   [0.         0.         0.         0.        ]]

  [[0.541478   0.19551845 0.         0.7155491 ]
   [0.4184445  0.         0.         0.        ]
   [0.         0.         0.         0.        ]
   [0.45869854

#### Extracting weights for 8x8 layer

In [5]:
w_bit = 4
weight_float_q = target_layer.weight_q
w_alpha = target_layer.weight_quant.wgt_alpha
w_delta = w_alpha / (2**(w_bit - 1)-1)

weight_int = (weight_float_q / w_delta)
print(f"Sample Weights (check if they're all integers for sanity): {weight_int[0][0][:][:]}")

weight_int = weight_int.round()
print(f"Sample Weights after rounding: {weight_int[0][0][:][:]}")

Sample Weights (check if they're all integers for sanity): tensor([[-1.0000, -1.0000, -2.0000],
        [-3.0000, -1.0000, -1.0000],
        [ 1.0000, -3.0000, -1.0000]], grad_fn=<SliceBackward0>)
Sample Weights after rounding: tensor([[-1., -1., -2.],
        [-3., -1., -1.],
        [ 1., -3., -1.]], grad_fn=<SliceBackward0>)


In [12]:
WGT_BITS = 4

weight_cpu = weight_int.detach().cpu().clone()  # [8, 8, 3, 3]
OC, IC, kH, kW = weight_cpu.shape
print("weight shape:", weight_cpu.shape)

assert OC == 8 and IC == 8 and kH == 3 and kW == 3

# col in your Verilog should be 8 (same as IC)
col = IC

# Enumerate kernel positions in some order; you can change naming if needed
kernel_positions = [(i, j) for i in range(3) for j in range(3)]

for idx, (kh, kw) in enumerate(kernel_positions):
    # Get the 8x8 weight matrix for this kernel tap
    # W_k[oc, ic] = weight for (out_channel, in_channel) at this (kh,kw)
    W_k = weight_cpu[:, :, kh, kw]   # [8, 8]

    filename = f"weight_itile0_otile0_kij{idx}.txt"   # k0..k8; rename if needed
    with open(filename, "w") as f:
        # No header lines here, because your Verilog uses "%32b" directly.
        # If you add comments, you must skip them in Verilog.

        # ic = input channel index, corresponds to t in Verilog loop
        for oc in range(OC):          # t = 0..7
            # Build one 32-bit word: row7..row0, each 4 bits
            line_bits = []
            for ic in range(IC):       # rows
                val = int(W_k[7 - oc, ic].item())  # row7 first, down to row0

                # Mask into 4 bits (unsigned view)
                val &= (1 << WGT_BITS) - 1
                bits = f"{val:0{WGT_BITS}b}"       # "0000".."1111"
                line_bits.append(bits)

            # Concatenate into 32-bit string
            f.write("".join(line_bits) + "\n")

    print(f"Saved {filename} for kernel tap (kh={kh}, kw={kw})")


weight shape: torch.Size([8, 8, 3, 3])
Saved weight_itile0_otile0_kij0.txt for kernel tap (kh=0, kw=0)
Saved weight_itile0_otile0_kij1.txt for kernel tap (kh=0, kw=1)
Saved weight_itile0_otile0_kij2.txt for kernel tap (kh=0, kw=2)
Saved weight_itile0_otile0_kij3.txt for kernel tap (kh=1, kw=0)
Saved weight_itile0_otile0_kij4.txt for kernel tap (kh=1, kw=1)
Saved weight_itile0_otile0_kij5.txt for kernel tap (kh=1, kw=2)
Saved weight_itile0_otile0_kij6.txt for kernel tap (kh=2, kw=0)
Saved weight_itile0_otile0_kij7.txt for kernel tap (kh=2, kw=1)
Saved weight_itile0_otile0_kij8.txt for kernel tap (kh=2, kw=2)


#### Extracting Inputs for 8x8 layer

In [6]:
x_bit = 4
x = conv8x8_input
x_alpha = target_layer.act_alpha.detach()
x_delta = x_alpha / (2**x_bit - 1)

x_div = x / x_alpha
x_clamped = torch.clamp(x_div, 0, 1)
x_int = (x_clamped * (2**x_bit - 1)).round()

print(f"Sample Inputs after quantization: {x_int[0][0][:][:]}")

Sample Inputs after quantization: tensor([[1., 3., 3., 3.],
        [0., 1., 0., 0.],
        [0., 0., 1., 1.],
        [0., 0., 0., 0.]])


In [ ]:
ACT_BITS = 4 # 4-bit activations

# Choose which sample in the batch you want to export
sample_id = 0   # 0..127
x_int_cpu = x_int.detach().cpu().clone()      # [128, 8, 4, 4]

# Take one sample: shape [8, 4, 4]
x_int_sample = x_int_cpu[sample_id]           # [C=8, H=4, W=4]

C, H, W = x_int_sample.shape
print("single sample shape:", x_int_sample.shape)
assert C == 8, f"Expected 8 channels, got {C}"
assert H * W == 16, f"Expected 4x4 spatial, got {H}x{W}"

# Flatten spatial into time: [rows=8, timesteps=16]
# Here: each channel → one systolic row, 16 time steps
X = x_int_sample.reshape(C, H * W)            # [8, 16]

with open('activation.txt', 'w') as f:
    f.write('#time0row7[msb-lsb],time0row6[msb-lsb],....,time0row0[msb-lsb]#\n')
    f.write('#time1row7[msb-lsb],time1row6[msb-lsb],....,time1row0[msb-lsb]#\n')
    f.write('#................#\n')

    # i: time step, j: row index
    for i in range(X.size(1)):       # time steps (0..15)
        for j in range(X.size(0)):   # row index (0..7)
            X_bin = '{0:04b}'.format(round(X[7-j,i].item()))
            #val = int(round(X[7 - j, i].item()))   # write row7..row0 order

            # If you're treating activations as unsigned 4-bit:
            #val = val & ((1 << ACT_BITS) - 1)

            #X_bin = f'{val:0{ACT_BITS}b}'         # "0000".."1111"
            for k in range(ACT_BITS):
                f.write(X_bin[k])
        f.write('\n')

print("Saved activation.txt")


single sample shape: torch.Size([8, 4, 4])
Saved activation.txt


#### Verifying Psum Integrity with reference psum

In [7]:
conv_int = nn.Conv2d(8, 8, kernel_size=3, padding=1, bias=False)
conv_int.weight = nn.Parameter(weight_int.float())
psum_int = conv_int(torch.tensor(x_int))
output_recovered = psum_int * x_delta * w_delta

act_fn = act_quantization(b=4)
x_model_q = act_fn(torch.tensor(x), torch.tensor(x_alpha))
output_ref = F.conv2d(torch.tensor(x_model_q), torch.tensor(target_layer.weight_q), bias=None, stride=1, padding=1)

difference = torch.abs(output_recovered - output_ref)
print(f"Mean Absolute Error: {difference.mean().item()}")
if difference.mean().item() < 1e-3:
    print("Success... Psum integrity verified!")
else:
    print("Error too high... Psum integrity failed.")

/tmp/ipykernel_346/2530370086.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  psum_int = conv_int(torch.tensor(x_int))
/tmp/ipykernel_346/2530370086.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_model_q = act_fn(torch.tensor(x), torch.tensor(x_alpha))


Mean Absolute Error: 9.626764949643984e-07
Success... Psum integrity verified!


/tmp/ipykernel_346/2530370086.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_ref = F.conv2d(torch.tensor(x_model_q), torch.tensor(target_layer.weight_q), bias=None, stride=1, padding=1)


In [ ]:
print("x_int shape:", x_int.shape)
print("weight_int shape:", weight_int.shape)
print("psum_int shape:", psum_int.shape)

In [11]:
PSUM_BITS = 16

sample_id = 0
p_sample = psum_int[sample_id]      # [8, 4, 4]
C, H, W = p_sample.shape
assert C == 8 and H == 4 and W == 4

# Flatten spatial dimension
P = p_sample.reshape(C, H*W)        # [8, 16]

with open("psum.txt", "w") as f:
    f.write("#time0row7[msb-lsb],time0row6[msb-lsb],...,time0row0[msb-lsb]#\n")
    f.write("#............................#\n")

    for t in range(P.size(1)):      # 16 time steps
        for r in range(P.size(0)):  # 8 rows
            val = int(P[7 - r, t].item())
            val &= (1 << PSUM_BITS) - 1

            bits = f"{val:0{PSUM_BITS}b}"
            f.write(bits)
        f.write("\n")

print("Saved psum.txt")


Saved psum.txt


In [12]:
PSUM_BITS = 16   # or match your RTL width

sample_id = 0
p_sample = psum_int[sample_id]      # shape [8,4,4]

C, H, W = p_sample.shape
assert C == 8 and H == 4 and W == 4

# Flatten spatial → time
P = p_sample.reshape(C, H*W)    # [8, 16]

with open("psum_relu.txt", "w") as f:
    f.write("#ReLUed psum output\n")
    f.write("#time0row7[msb-lsb],time0row6[msb-lsb],...,row0#\n")
    f.write("#....................................#\n")

    for t in range(P.size(1)):      # 16 timesteps
        for r in range(P.size(0)):  # 8 rows
            val = int(P[7-r,t].item())

            # Apply ReLU
            val = max(0, val)

            # Fit into chosen bit width
            val &= (1 << PSUM_BITS) - 1

            bits = f"{val:0{PSUM_BITS}b}"
            f.write(bits)

        f.write("\n")

print("Saved psum_relu.txt")


Saved psum_relu.txt


In [13]:
PSUM_BITS = 16
sample_id = 0

x_sample = x_int[sample_id].detach().cpu().clone()      # [8,4,4]
w_cpu    = weight_int.detach().cpu().clone()            # [8,8,3,3]

OC, IC, kH, kW = w_cpu.shape
C, H, W = x_sample.shape
assert H==4 and W==4 and C==8

kernel_positions = [(i,j) for i in range(3) for j in range(3)]

# pad input (padding=1)
x_pad = torch.zeros((C, H+2, W+2), dtype=torch.int32)
x_pad[:,1:H+1,1:W+1] = x_sample

for idx, (kh, kw) in enumerate(kernel_positions):
    # Extract weight matrix for this tap
    W_k = w_cpu[:,:,kh,kw]   # [8,8]

    # Output psum for this kernel (8,4,4)
    psum_k = torch.zeros((OC,H,W), dtype=torch.int32)

    # Compute per-tap convolution manually
    for oc in range(OC):
        for ic in range(IC):
            w_val = int(W_k[oc, ic].item())
            for h in range(H):
                for w in range(W):
                    # shifted input index = (h + kh, w + kw)
                    xi = x_pad[ic, h + kh, w + kw].item()
                    psum_k[oc, h, w] += xi * w_val

    # Now dump as file with 16-bit mask
    filename = f"psum_{idx}.txt"
    with open(filename, "w") as f:
        for t in range(H*W):         # 16 timesteps
            h = t // W
            w = t %  W

            line_bits = []
            for oc in range(OC):    # row7..row0
                val = int(psum_k[7 - oc, h, w].item())
                val &= (1 << PSUM_BITS) - 1
                bits = f"{val:0{PSUM_BITS}b}"
                line_bits.append(bits)

            f.write("".join(line_bits) + "\n")

    print("Saved", filename)

Saved psum_0.txt
Saved psum_1.txt
Saved psum_2.txt
Saved psum_3.txt
Saved psum_4.txt
Saved psum_5.txt
Saved psum_6.txt
Saved psum_7.txt
Saved psum_8.txt


#### Verifying Psum Integrity with next layer input

In [9]:
difference2 = torch.abs(input_records['next_layer_input'].cpu() - output_recovered)
print(f"Mean Absolute Error with next layer input: {difference2.mean().item()}")
if difference2.mean().item() < 1e-3:
    print("Success... Psum integrity verified!")
else:
    print("Error too high... Psum integrity failed.")

Mean Absolute Error with next layer input: 1.4162951629259624e-06
Success... Psum integrity verified!


# ECE 284 Final Project - Software

## Vanilla Training VGGNet with 2 bit activation and 4 bit weight quantisation

### Initialisation Code

In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms

from models import *

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')

batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant(wbit=4, abit=2)

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])

train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
    
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

### Loading Pre trained w4a4 model for faster convergence

In [2]:
def load_checkpoint_special(model, checkpoint_path):
    print(f"=> Loading checkpoint '{checkpoint_path}'")
    checkpoint = torch.load(checkpoint_path)
    pretrained_dict = checkpoint['state_dict']
    model_dict = model.state_dict()
    pretrained_dict_filtered = {}
    for k, v in pretrained_dict.items():
        if k in model_dict:
            if model_dict[k].shape == v.shape:
                pretrained_dict_filtered[k] = v
            else:
                print(f"\tSkipping layer '{k}': Shape mismatch. "
                    f"New: {model_dict[k].shape} vs Old: {v.shape}")
        else:
            print(f"\tSkipping layer '{k}': Not found in new model.")
    model_dict.update(pretrained_dict_filtered) 
    model.load_state_dict(model_dict)
    print("=> Partial load complete. Mismatched layers initialized randomly.")

load_checkpoint_special(model, './result/VGG16_4a4w_quant_noskipblock/checkpoint.pth')

=> Loading checkpoint './result/VGG16_4a4w_quant_noskipblock/checkpoint.pth'


/tmp/ipykernel_1037538/1623649740.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


	Skipping layer 'features.24.weight': Shape mismatch. New: torch.Size([512, 256, 3, 3]) vs Old: torch.Size([16, 256, 3, 3])
	Skipping layer 'features.24.weight_q': Shape mismatch. New: torch.Size([512, 256, 3, 3]) vs Old: torch.Size([16, 256, 3, 3])
	Skipping layer 'features.25.weight': Shape mismatch. New: torch.Size([512]) vs Old: torch.Size([16])
	Skipping layer 'features.25.bias': Shape mismatch. New: torch.Size([512]) vs Old: torch.Size([16])
	Skipping layer 'features.25.running_mean': Shape mismatch. New: torch.Size([512]) vs Old: torch.Size([16])
	Skipping layer 'features.25.running_var': Shape mismatch. New: torch.Size([512]) vs Old: torch.Size([16])
	Skipping layer 'features.27.weight': Shape mismatch. New: torch.Size([512, 512, 3, 3]) vs Old: torch.Size([16, 16, 3, 3])
	Skipping layer 'features.27.weight_q': Shape mismatch. New: torch.Size([512, 512, 3, 3]) vs Old: torch.Size([16, 16, 3, 3])
	Skipping layer 'features.30.weight': Shape mismatch. New: torch.Size([512, 512, 3, 3

### Chokepoint Block and SkipBlock Optimisation

In [3]:
# [SEE custom_vgg.py for details on SkipBlock optimisation]
from models.custom_vgg import SkipBlock

use_skip_block = False

model_name = "VGG16_2a4w_quant_skipblock" if use_skip_block else "VGG16_2a4w_quant_noskipblock"

if use_skip_block:
    print("Using SkipBlock optimisation")
    skip_block = SkipBlock()
    model.features[20] = skip_block
    for i in range(21, 33):
        model.features[i] = nn.Identity()
else:
    print("Not using SkipBlock optimisation")
    model.features[24] = QuantConv2d(256, 16, kernel_size=3, padding=1, bias=False)
    model.features[25] = nn.BatchNorm2d(16)
    model.features[26] = nn.ReLU(inplace=True)
    
    model.features[27] = QuantConv2d(16, 16, kernel_size=3, padding=1, bias=False)
    model.features[28] = nn.Identity()
    model.features[29] = nn.ReLU(inplace=True)
    
    model.features[30] = QuantConv2d(16, 512, kernel_size=3, padding=1, bias=False)
    model.features[31] = nn.BatchNorm2d(512)
    model.features[32] = nn.ReLU(inplace=True)

print(model)

Not using SkipBlock optimisation
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3

### Training Loop

In [16]:
lr = 1e-2
weight_decay = 1e-5
epochs = 300
best_prec = 0

model.cuda()

all_params = model.named_parameters()
params_main = []
params_quant = []

for name, param in all_params:
    if 'alpha' in name:
        params_quant.append(param)
    else:
        params_main.append(param)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1).cuda()
optimizer = torch.optim.SGD([
    {'params': params_main, 'weight_decay': weight_decay, 'lr': lr},
    {'params': params_quant, 'weight_decay': 0.0, 'lr': lr * 0.1}
], momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)

for epoch in range(0, epochs):
    # adjust_learning_rate(optimizer, epoch)
    train(trainloader, model, criterion, optimizer, epoch)
    scheduler.step()

    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)
    # if best_prec > 87.0:
    #     print("Achieved accuracy greater than 87%, early stopping.")
    #     break

Epoch: [0][0/391]	Time 0.174 (0.174)	Data 0.130 (0.130)	Loss 3.1802 (3.1802)	Prec 10.156% (10.156%)
Epoch: [0][100/391]	Time 0.035 (0.034)	Data 0.002 (0.003)	Loss 1.6310 (2.0794)	Prec 48.438% (26.911%)
Epoch: [0][200/391]	Time 0.034 (0.034)	Data 0.002 (0.003)	Loss 1.5967 (1.8636)	Prec 52.344% (37.072%)
Epoch: [0][300/391]	Time 0.034 (0.033)	Data 0.002 (0.002)	Loss 1.6274 (1.7414)	Prec 52.344% (43.161%)
Validation starts
Test: [0/79]	Time 0.114 (0.114)	Loss 1.4094 (1.4094)	Prec 60.156% (60.156%)
 * Prec 61.730% 
best acc: 61.730000
Epoch: [1][0/391]	Time 0.189 (0.189)	Data 0.149 (0.149)	Loss 1.3061 (1.3061)	Prec 65.625% (65.625%)
Epoch: [1][100/391]	Time 0.030 (0.035)	Data 0.002 (0.004)	Loss 1.3399 (1.3717)	Prec 62.500% (61.680%)
Epoch: [1][200/391]	Time 0.036 (0.034)	Data 0.002 (0.003)	Loss 1.2620 (1.3457)	Prec 68.750% (62.924%)
Epoch: [1][300/391]	Time 0.027 (0.033)	Data 0.002 (0.003)	Loss 1.2681 (1.3310)	Prec 65.625% (63.626%)
Validation starts
Test: [0/79]	Time 0.109 (0.109)	Loss 1.

### Testing Loop

In [4]:
PATH = "result/VGG16_2a4w_quant_noskipblock/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

/tmp/ipykernel_1037538/3160766666.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)



Test set: Accuracy: 8802/10000 (88%)



### Weight and Activation recovery verification

#### Getting the right target layer and attaching hooks for fwd pass

In [5]:
if use_skip_block:
    target_layer = model.features[20].conv27
    next_layer = model.features[20].bn28
else:
    target_layer = model.features[27]
    next_layer = model.features[28]

# some sanity checking to ensure we're extracting the right layer
assert isinstance(target_layer, QuantConv2d)

input_records = {}
def get_input_hook(name):
    def hook(model, input, output):
        input_records[name] = input[0].detach().clone()
    return hook

target_layer.register_forward_hook(get_input_hook('conv16x16_input'))
next_layer.register_forward_hook(get_input_hook('next_layer_input'))

# Running a sample forward pass to extract inputs
data, _ = next(iter(testloader))
data = data.cuda()
model = model.cuda()
model.eval()
with torch.no_grad():
    output = model(data)
conv16x16_input = input_records['conv16x16_input'].cpu().numpy()
print(f"Captured Transient Input for 16x16 layer: {conv16x16_input}")

target_layer = target_layer.cpu()

Captured Transient Input for 16x16 layer: [[[[1.23831189e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 6.71324193e-01 0.00000000e+00]
   [0.00000000e+00 1.32691607e-01 1.76276970e+00 7.28025019e-01]
   [5.15404284e-01 1.03986478e+00 1.73442006e+00 1.47927773e+00]]

  [[1.89826801e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 1.77257344e-01 4.03556108e-01 1.32132292e+00]
   [0.00000000e+00 0.00000000e+00 4.41273540e-01 0.00000000e+00]]

  [[1.32021332e+00 1.73326778e+00 2.71155310e+00 3.33113289e+00]
   [0.00000000e+00 1.26586592e+00 2.74416304e+00 3.22243619e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
   [6.13675117e-01 0.00000000e+00 1.01585865e+00 1.68978882e+00]]

  ...

  [[0.00000000e+00 7.08765149e-01 1.70264459e+00 9.02694762e-01]
   [2.22382259e+00 4.81758881e+00 6.25991678e+00 4.12672472e+00]
   [3.03589177e+00 3.92067838e+00 6

#### Extracting weights for 16x16 layer

In [6]:
w_bit = 4
weight_float_q = target_layer.weight_q
w_alpha = target_layer.weight_quant.wgt_alpha
w_delta = w_alpha / (2**(w_bit - 1)-1)

weight_int = (weight_float_q / w_delta)
print(f"Sample Weights (check if they're all integers for sanity): {weight_int[0][0][:][:]}")

weight_int = weight_int.round()
print(f"Sample Weights after rounding: {weight_int[0][0][:][:]}")

Sample Weights (check if they're all integers for sanity): tensor([[-0., 3., 2.],
        [1., 2., 2.],
        [3., 3., 2.]], grad_fn=<SliceBackward0>)
Sample Weights after rounding: tensor([[-0., 3., 2.],
        [1., 2., 2.],
        [3., 3., 2.]], grad_fn=<SliceBackward0>)


#### Extracting Inputs for 16x16 layer

In [7]:
x_bit = 2
x = conv16x16_input
x_alpha = target_layer.act_alpha.detach()
x_delta = x_alpha / (2**x_bit - 1)

x_div = x / x_alpha
x_clamped = torch.clamp(x_div, 0, 1)
x_int = (x_clamped * (2**x_bit - 1)).round()

print(f"Sample Inputs after quantization: {x_int[0][0][:][:]}")

Sample Inputs after quantization: tensor([[1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 1., 1.]])


#### Verifying Psum Integrity with reference psum

In [8]:
conv_int = nn.Conv2d(16, 16, kernel_size=3, padding=1, bias=False)
conv_int.weight = nn.Parameter(weight_int.float())
psum_int = conv_int(torch.tensor(x_int))
output_recovered = psum_int * x_delta * w_delta

act_fn = act_quantization(b=2)
x_model_q = act_fn(torch.tensor(x), torch.tensor(x_alpha))
output_ref = F.conv2d(torch.tensor(x_model_q), torch.tensor(target_layer.weight_q), bias=None, stride=1, padding=1)

difference = torch.abs(output_recovered - output_ref)
print(f"Mean Absolute Error: {difference.mean().item()}")
if difference.mean().item() < 1e-3:
    print("Success... Psum integrity verified!")
else:
    print("Error too high... Psum integrity failed.")

Mean Absolute Error: 1.5899438494670903e-06
Success... Psum integrity verified!


/tmp/ipykernel_1037538/3644988574.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  psum_int = conv_int(torch.tensor(x_int))
/tmp/ipykernel_1037538/3644988574.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_model_q = act_fn(torch.tensor(x), torch.tensor(x_alpha))
/tmp/ipykernel_1037538/3644988574.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_ref = F.conv2d(torch.tensor(x_model_q), torch.tensor(target_layer.weight_q), bias=None, stride=1, padding=1)


#### Verifying Psum Integrity with next layer input

In [9]:
difference2 = torch.abs(input_records['next_layer_input'].cpu() - output_recovered)
print(f"Mean Absolute Error with next layer input: {difference2.mean().item()}")
if difference2.mean().item() < 1e-3:
    print("Success... Psum integrity verified!")
else:
    print("Error too high... Psum integrity failed.")

Mean Absolute Error with next layer input: 1.6895540966288536e-06
Success... Psum integrity verified!
